In [1]:
# Import libraries
import os
import requests
from zipfile import ZipFile
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from datetime import datetime
import time
import glob
import contextlib
from PIL import Image

# Main data source
# https://gbfs.nextbike.net/maps/gbfs/v1/nextbike_gg/gbfs.json
# Updates every ~1 minute

In [16]:
# Set interval (minutes)
interval = 10

# Set runtime (minutes)
runtime = 1500

# Calculate interval in seconds and number of iterations to perform
iterations = int(runtime/interval)
interval = interval*60

In [3]:
# Create output folders
try:
    os.mkdir('glasgow-data')
except:
    print('Data folder already exists')
try:
    os.mkdir('glasgow-plots')
except:
    print('Plots folder already exists')

Data folder already exists
Plots folder already exists


In [4]:
# Request JSON for station information
stationsurl = 'https://gbfs.nextbike.net/maps/gbfs/v1/nextbike_gg/en/station_information.json'
req = requests.get(stationsurl)
print(req)

# Extract stations from JSON
jsondata = json.loads(req.text)
stations = jsondata['data']['stations']

# Convert JSON to dataframe
data = []
for station in stations:
    stationdata = []
    stationdata.append(station['station_id'])
    stationdata.append(station['name'])
    stationdata.append(station['short_name'])
    stationdata.append(station['lat'])
    stationdata.append(station['lon'])
    stationdata.append(station['region_id'])
    try:
        stationdata.append(station['capacity'])
    except:
        stationdata.append(4)
    data.append(stationdata)
    
data = pd.DataFrame(data, columns=['station_id', 'name', 'short_name', 'lat', 'lon', 'region_id', 'capacity'])

# Convert to geodataframe
geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]
data = gpd.GeoDataFrame(data, crs='epsg:4326', geometry=geometry)
data.head()

<Response [200]>


,station_id,name,short_name,lat,lon,region_id,capacity,geometry
0,264283,Waterloo Street - ELECTRIC,8402,55.860770,-4.264535,237,4,POINT (-4.26454 55.86077)
1,264292,George Square - ELECTRIC,8406,55.861550,-4.249400,237,10,POINT (-4.24940 55.86155)
2,264293,Merchant Square - ELECTRIC,8407,55.858167,-4.245483,237,8,POINT (-4.24548 55.85817)
3,264294,St George's Cross - ELECTRIC,8408,55.871367,-4.269117,237,10,POINT (-4.26912 55.87137)
4,264295,St Enoch Square - ELECTRIC,8410,55.856859,-4.255593,237,10,POINT (-4.25559 55.85686)


In [ ]:
# Download map data (Approx. 400MB)
mapurl = 'http://download.geofabrik.de/europe/great-britain/scotland-latest-free.shp.zip'
req = requests.get(mapurl, allow_redirects=True)
open('scotland-latest-free.shp.zip', 'wb').write(req.content)

In [ ]:
# Extract necessary files from downloaded zip file
with ZipFile('scotland-latest-free.shp.zip', 'r') as zObject:
    zObject.extract('gis_osm_roads_free_1.cpg', path='scotland-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.dbf', path='scotland-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.prj', path='scotland-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.shp', path='scotland-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.shx', path='scotland-latest-free.shp')
zObject.close()

In [5]:
# Read roads data
roads = gpd.read_file('scotland-latest-free.shp/gis_osm_roads_free_1.shp')

# Filter minor roads
roads = roads[~roads['fclass'].isin(['residential', 'footway', 'cycleway', 'bridleway', 'busway',
                                     'track_grade1', 'track_grade2', 'track_grade3', 'track_grade4',
                                     'track_grade5', 'track_grade6', 'path', 'track', 'steps', 'unknown',
                                     'service'])]
roads.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
1,370,5131,motorway_link,None,M9,F,112,0,F,F,"LINESTRING (-3.41038 55.94936, -3.40838 55.94752)"
2,375,5131,motorway_link,None,M8,F,112,0,F,F,"LINESTRING (-3.30997 55.92267, -3.31006 55.922..."
19,1665,5135,tertiary_link,City of Edinburgh Bypass,A720,F,64,0,F,F,"LINESTRING (-3.31948 55.93824, -3.32022 55.938..."
20,1708,5131,motorway_link,None,M8,F,80,0,F,F,"LINESTRING (-3.40070 55.93516, -3.40098 55.935..."
21,1709,5111,motorway,None,M9,F,80,0,F,F,"LINESTRING (-3.40070 55.93516, -3.40084 55.935..."


In [6]:
# Function to fetch data and save as csv file
def fetchdata(data):
    # Request JSON
    liveurl = 'https://gbfs.nextbike.net/maps/gbfs/v1/nextbike_gg/en/station_status.json'
    req = requests.get(liveurl)

    # Generate filename from timestamp
    now = datetime.now()
    filename = 'glasgow-data/' + now.strftime("%Y%m%d %H%M%S") + '.csv'

    # Extract stations from JSON
    jsondata = json.loads(req.text)
    stations = jsondata['data']['stations']

    # Convert JSON to dataframe
    livedata = []
    for station in stations:
        stationdata = []
        stationdata.append(station['station_id'])
        stationdata.append(station['num_bikes_available'])
        stationdata.append(station['num_docks_available'])
        stationdata.append(station['is_installed'])
        stationdata.append(station['is_renting'])
        stationdata.append(station['last_reported'])
        livedata.append(stationdata)

    livedata = pd.DataFrame(livedata, columns=['station_id', 'num_bikes_available', 'num_docks_available', 
                                               'is_installed', 'is_renting', 'last_reported'])
    livedata = livedata[['station_id', 'num_bikes_available', 'num_docks_available']]

    # Merge livedata with stationsdata, calculate fill percent for each station
    livedata = livedata.merge(data, on='station_id')
    livedata['fillpercent'] = livedata['num_bikes_available']/livedata['num_docks_available']*100
    livedata['fillpercent'] = livedata['fillpercent'].clip(upper=100)
    livedata.to_csv(filename)

In [14]:
# Function to plot data
def plotdata(filename, roads, livedata):
    # Calculate timestamp
    timestamp = filename[23:-8] + ':' + filename[25:-6]
    
    # Plot station fill percentages, save image
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8), gridspec_kw={'width_ratios': [30, 1]})
    plt.subplots_adjust(wspace=0, hspace=0)

    roads.plot(ax=ax1, alpha=0.3, color='grey')
    livedata.plot(markersize=(livedata['num_bikes_available']*20), ax=ax1)
    ax1.set_xlim(-4.375, -4.125)
    ax1.set_ylim(55.81, 55.9)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('Ovo Bikes Available in Glasgow. Time: ' + timestamp)

    ax2.bar([1], [livedata['num_bikes_available'].mean()])
    ax2.set_ylim(0, 10)
    ax2.set_xticks([])
    ax2.set_ylabel('Mean Bikes Available')

    fig.savefig(filename, dpi=200, bbox_inches='tight')
    plt.close()

In [ ]:
# Fetch data with provided interval and runtime
datapoints = 0
while datapoints < iterations:
    fetchdata(data)
    datapoints += 1
    time.sleep(interval - time.time() % interval)
    print(str(datapoints) + '/' + str(iterations) + ' iterations complete')

1/150 iterations complete
2/150 iterations complete
3/150 iterations complete
4/150 iterations complete
5/150 iterations complete
6/150 iterations complete
7/150 iterations complete
8/150 iterations complete
9/150 iterations complete
10/150 iterations complete
11/150 iterations complete
12/150 iterations complete
13/150 iterations complete
14/150 iterations complete
15/150 iterations complete
16/150 iterations complete
17/150 iterations complete
18/150 iterations complete
19/150 iterations complete
20/150 iterations complete
21/150 iterations complete
22/150 iterations complete
23/150 iterations complete
24/150 iterations complete
25/150 iterations complete
26/150 iterations complete
27/150 iterations complete
28/150 iterations complete
29/150 iterations complete
30/150 iterations complete
31/150 iterations complete
32/150 iterations complete
33/150 iterations complete
34/150 iterations complete
35/150 iterations complete
36/150 iterations complete
37/150 iterations complete
38/150 ite

In [15]:
# Plot images from csvs
csvlist = os.listdir('glasgow-data')
plotted = 0
for csv in csvlist:
    plotted += 1
    filename = 'glasgow-data/' + csv
    livedata = pd.read_csv(filename, index_col=0)
    livedata = gpd.GeoDataFrame(livedata, crs='epsg:4326', geometry=geometry)
    filename = 'glasgow-plots/' + csv[:-4] + '.png'
    plotdata(filename, roads, livedata)
    print(str(plotted) + '/' + str(len(csvlist)) + ' iterations complete')

1/6 iterations complete
2/6 iterations complete
3/6 iterations complete
4/6 iterations complete
5/6 iterations complete
6/6 iterations complete


In [12]:
# Combine images to gif output
# Filepaths
fp_in = 'glasgow-plots/*.png'
fp_out = 'glasgow-plots/glasgow-bike-movements.gif'

# Use ExitStack to automatically close opened images
with contextlib.ExitStack() as stack:
    # Load images
    images = (stack.enter_context(Image.open(f)) for f in sorted(glob.glob(fp_in)))
    # Extract first image from iterator
    image = next(images)
    # Save as gif
    image.save(fp=fp_out, format='gif', append_images=images, save_all=True, duration=100, loop=0)